# Tutorial: Creating a Hybrid Retrieval Pipeline

- **Level**: Intermediate
- **Time to complete**: 15 minutes
- **Components Used**: [`DocumentSplitter`](https://docs.haystack.deepset.ai/docs/documentsplitter), [`SentenceTransformersDocumentEmbedder`](https://docs.haystack.deepset.ai/docs/sentencetransformersdocumentembedder), [`DocumentJoiner`](https://docs.haystack.deepset.ai/docs/documentjoiner), [`InMemoryDocumentStore`](https://docs.haystack.deepset.ai/docs/inmemorydocumentstore), [`InMemoryBM25Retriever`](https://docs.haystack.deepset.ai/docs/inmemorybm25retriever), [`InMemoryEmbeddingRetriever`](https://docs.haystack.deepset.ai/docs/inmemoryembeddingretriever), and [`TransformersSimilarityRanker`](https://docs.haystack.deepset.ai/docs/transformerssimilarityranker)
- **Prerequisites**: None
- **Goal**: After completing this tutorial, you will have learned about creating a hybrid retrieval and when it's useful.

In [1]:
import os
import sys
import warnings

import pandas as pd

from tqdm import tqdm
from uuid import uuid4

from langchain_chroma import Chroma
# from langchain_huggingface import HuggingFaceEmbeddings

# from langchain_core.documents import Document

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

# import log_files
from data_processing import DataProcessing
from text_generation_models import TextGenerationModelFactory

In [2]:
pd.set_option('max_colwidth', 800)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## Overview

**Hybrid Retrieval** combines keyword-based and embedding-based retrieval techniques, leveraging the strengths of both approaches. In essence, dense embeddings excel in grasping the contextual nuances of the query, while keyword-based methods excel in matching keywords.

There are many cases when a simple keyword-based approaches like BM25 performs better than a dense retrieval (for example in a specific domain like healthcare) because a dense model needs to be trained on data. For more details about Hybrid Retrieval, check out [Blog Post: Hybrid Document Retrieval](https://haystack.deepset.ai/blog/hybrid-retrieval).

## Preparing the Colab Environment

- [Enable GPU Runtime in Colab](https://docs.haystack.deepset.ai/docs/enabling-gpu-acceleration)
- [Set logging level to INFO](https://docs.haystack.deepset.ai/docs/setting-the-log-level)

## Installing Haystack

Install Haystack and other required packages with `pip`:

In [3]:
# %%bash

# pip install haystack-ai
# pip install "datasets>=2.6.1"
# pip install "sentence-transformers>=4.1.0"
# pip install accelerate

## Initializing the DocumentStore

You'll start creating your question answering system by initializing a DocumentStore. A DocumentStore stores the Documents that your system uses to find answers to your questions. In this tutorial, you'll be using the [`InMemoryDocumentStore`](https://docs.haystack.deepset.ai/docs/inmemorydocumentstore).

In [4]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

> `InMemoryDocumentStore` is the simplest DocumentStore to get started with. It requires no external dependencies and it's a good option for smaller projects and debugging. But it doesn't scale up so well to larger Document collections, so it's not a good choice for production systems. To learn more about the different types of external databases that Haystack supports, see [DocumentStore Integrations](https://haystack.deepset.ai/integrations?type=Document+Store&version=2.0).

## Fetching and Processing Documents

As Documents, you will use the PubMed Abstracts. There are a lot of datasets from PubMed on Hugging Face Hub; you will use [anakin87/medrag-pubmed-chunk](https://huggingface.co/datasets/anakin87/medrag-pubmed-chunk) in this tutorial.

Then, you will create Documents from the dataset with a simple for loop.
Each data point in the PubMed dataset has 4 features:
* *pmid*
* *title*
* *content*: the abstract
* *contents*: abstract + title

For searching, you will use the *contents* feature. The other features will be stored as metadata, and you will use them to have a **pretty print** of the search results or for [metadata filtering](https://docs.haystack.deepset.ai/docs/metadata-filtering).

In [5]:
from datasets import load_dataset
from haystack import Document

# dataset = load_dataset("anakin87/medrag-pubmed-chunk", split="train")
# base_path = os.path.join('/orange/ufdatastudios/dj.brinkley/predictions/data/')
financial_full_path = os.path.join(notebook_dir, '../data/', 'financial_phrase_bank/all_data-adjusted_header.csv')

dataset = load_dataset("csv", data_files=financial_full_path, split="train")

docs = []
# for doc in dataset:
#     docs.append(
#         Document(content=doc["contents"], meta={"title": doc["title"], "abstract": doc["content"], "pmid": doc["id"]})
#     )
for doc in dataset:
    docs.append(
        Document(content=doc["sentence"], meta={"sentiment": doc["sentiment"]})
    )

In [6]:
docs = docs[:33]
docs

[Document(id=63461e5f36b506ade08357e3bdb34e98c8ba387ec40e2d27f0b651cdc0416edf, content: 'According to Gran , the company has no plans to move all production to Russia , although that is whe...', meta: {'sentiment': 'neutral'}),
 Document(id=99e7dc0575e03709f270d73b2dd4f9b878b6e2b998cdc488afb27daad6fb44a2, content: 'Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to hos...', meta: {'sentiment': 'neutral'}),
 Document(id=a733e18a009c0cdef1ee85366b3506eb3f869588c69b0c4d604021aa6d34a1df, content: 'The international electronic industry company Elcoteq has laid off tens of employees from its Tallin...', meta: {'sentiment': 'negative'}),
 Document(id=39e2ff2d164bed02386d1da93730042991b977cb8b62db9d4786722a19a10695, content: 'With the new production plant the company would increase its capacity to meet the expected increase ...', meta: {'sentiment': 'positive'}),
 Document(id=1f38a8a49050433e5fabd819b0bc84b4944a4f683c6797d3fe98a0feeb0c8146, content

## Indexing Documents with a Pipeline

Create a pipeline to store the data in the document store with their embedding. For this pipeline, you need a [DocumentSplitter](https://docs.haystack.deepset.ai/docs/documentsplitter) to split documents into chunks of 512 words, [SentenceTransformersDocumentEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformersdocumentembedder) to create document embeddings for dense retrieval and [DocumentWriter](https://docs.haystack.deepset.ai/docs/documentwriter) to write documents to the document store.

As an embedding model, you will use [BAAI/bge-small-en-v1.5](https://huggingface.co/BAAI/bge-small-en-v1.5) on Hugging Face. Feel free to test other models on Hugging Face or use another [Embedder](https://docs.haystack.deepset.ai/docs/embedders) to switch the model provider.

> If this step takes too long for you, replace the embedding model with a smaller model such as `sentence-transformers/all-MiniLM-L6-v2` or `sentence-transformers/all-mpnet-base-v2`. Make sure that the `split_length` is updated according to your model's token limit.

In [7]:
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.preprocessors.document_splitter import DocumentSplitter
from haystack import Pipeline
from haystack.utils import ComponentDevice

document_splitter = DocumentSplitter(split_by="word", split_length=512, split_overlap=32)
document_embedder = SentenceTransformersDocumentEmbedder(
    model="BAAI/bge-small-en-v1.5"
)
document_writer = DocumentWriter(document_store)

indexing_pipeline = Pipeline()
indexing_pipeline.add_component("document_splitter", document_splitter)
indexing_pipeline.add_component("document_embedder", document_embedder)
indexing_pipeline.add_component("document_writer", document_writer)

indexing_pipeline.connect("document_splitter", "document_embedder")
indexing_pipeline.connect("document_embedder", "document_writer")

indexing_pipeline.run({"document_splitter": {"documents": docs}})

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

{'document_writer': {'documents_written': 33}}

Documents are stored in `InMemoryDocumentStore` with their embeddings, now it's time for creating the hybrid retrieval pipeline ✅

## Creating a Pipeline for Hybrid Retrieval

Hybrid retrieval refers to the combination of multiple retrieval methods to enhance overall performance. In the context of search systems, a hybrid retrieval pipeline executes both traditional keyword-based search and dense vector search, later ranking the results with a cross-encoder model. This combination allows the search system to leverage the strengths of different approaches, providing more accurate and diverse results.

Here are the required steps for a hybrid retrieval pipeline:

### 1) Initialize Retrievers and the Embedder

Initialize a [InMemoryEmbeddingRetriever](https://docs.haystack.deepset.ai/docs/inmemoryembeddingretriever) and [InMemoryBM25Retriever](https://docs.haystack.deepset.ai/docs/inmemorybm25retriever) to perform both dense and keyword-based retrieval. For dense retrieval, you also need a [SentenceTransformersTextEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformerstextembedder) that computes the embedding of the search query by using the same embedding model `BAAI/bge-small-en-v1.5` that was used in the indexing pipeline:

In [8]:
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever, InMemoryEmbeddingRetriever
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder(
    model="BAAI/bge-small-en-v1.5"
)
embedding_retriever = InMemoryEmbeddingRetriever(document_store)
bm25_retriever = InMemoryBM25Retriever(document_store)

### 2) Join Retrieval Results

Haystack offers several joining methods in [`DocumentJoiner`](https://docs.haystack.deepset.ai/docs/documentjoiner) to be used for different use cases such as `merge` and `reciprocal_rank_fusion`. In this example, you will use the default `concatenate` mode to join the documents coming from two Retrievers as the [Ranker](https://docs.haystack.deepset.ai/docs/rankers) will be the main component to rank the documents for relevancy.

In [9]:
from haystack.components.joiners import DocumentJoiner

document_joiner = DocumentJoiner()

### 3) Rank the Results

Use the [TransformersSimilarityRanker](https://docs.haystack.deepset.ai/docs/transformerssimilarityranker) that scores the relevancy of all retrieved documents for the given search query by using a cross encoder model. In this example, you will use [BAAI/bge-reranker-base](https://huggingface.co/BAAI/bge-reranker-base) model to rank the retrieved documents but you can replace this model with other cross-encoder models on Hugging Face.

In [10]:
from haystack.components.rankers import TransformersSimilarityRanker

ranker = TransformersSimilarityRanker(model="BAAI/bge-reranker-base")

TransformersSimilarityRanker is considered legacy and will no longer receive updates. It may be deprecated in a future release, with removal following after a deprecation period. Consider using SentenceTransformersSimilarityRanker instead, which provides the same functionality along with additional features.


### 4) Create the Hybrid Retrieval Pipeline

Add all initialized components to your pipeline and connect them.

In [11]:
from haystack import Pipeline

hybrid_retrieval = Pipeline()
hybrid_retrieval.add_component("text_embedder", text_embedder)
hybrid_retrieval.add_component("embedding_retriever", embedding_retriever)
hybrid_retrieval.add_component("bm25_retriever", bm25_retriever)
hybrid_retrieval.add_component("document_joiner", document_joiner)
hybrid_retrieval.add_component("ranker", ranker)

hybrid_retrieval.connect("text_embedder", "embedding_retriever")
hybrid_retrieval.connect("bm25_retriever", "document_joiner")
hybrid_retrieval.connect("embedding_retriever", "document_joiner")
hybrid_retrieval.connect("document_joiner", "ranker")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - embedding_retriever: InMemoryEmbeddingRetriever
  - bm25_retriever: InMemoryBM25Retriever
  - document_joiner: DocumentJoiner
  - ranker: TransformersSimilarityRanker
🛤️ Connections
  - text_embedder.embedding -> embedding_retriever.query_embedding (list[float])
  - embedding_retriever.documents -> document_joiner.documents (list[Document])
  - bm25_retriever.documents -> document_joiner.documents (list[Document])
  - document_joiner.documents -> ranker.documents (list[Document])

### 5) Visualize the Pipeline (Optional)

To understand how you formed a hybrid retrieval pipeline, use [draw()](https://docs.haystack.deepset.ai/docs/drawing-pipeline-graphs) method of the pipeline. If you're running this notebook on Google Colab, the generate file will be saved in "Files" section on the sidebar.

In [12]:
# hybrid_retrieval.draw("hybrid-retrieval.png")

## Testing the Hybrid Retrieval

Pass the query to `text_embedder`, `bm25_retriever` and `ranker` and run the retrieval pipeline:


In [13]:
prediction_properties = DataProcessing.load_prediction_properties()
prediction_properties

' A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:\n\n        1. <p_s>\n            - Defined as: \n                - Source entity that states the <p>\n            - Characteristics:\n                - A person with either: a name only, profile name only, geneder only, domain specific title only or any combination of these.\n                - An associated organization\n                - Named entity: Person, organization\n                - Part of speech: Noun\n            - Examples:\n                1. A person with a name only: Detravious\n                2. A person with a profile name: FitToJesus\n                3. A person with a gender only: He\n                4. A person with a domain specific title: reporter, analyst, expert, top executive, senior level person, etc \n                5. A person with a combination: Detravious, a reporter\n                6. An associated organization: FitTo...\n                7. A combina

In [14]:
future_verbs = [
    "will",
    "shall",
    "would",
    "going",
    "might",
    "should",
    "could",
    "may",
    "must",
    "can"
]
query = f"""Can you identify the predictions from the documents? I define a prediction as: {prediction_properties}. Note that the documents should be future tense like {future_verbs}. \
    
    Some examples of predictions in the PhraseBank dataset are \n
        1. According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing . \n
        2. According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales .
        3. Its board of directors will propose a dividend of EUR0 .12 per share for 2010 , up from the EUR0 .08 per share paid in 2009 .
    Some examples of non-predictions in the P
        1. Net sales increased to EUR193 .3 m from EUR179 .9 m and pretax profit rose by 34.2 % to EUR43 .1 m. ( EUR1 = USD1 .4 )
        2. Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .
        3. STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) raised the fair value for shares in four of the largest Nordic forestry groups .
"""

retrieved_result = hybrid_retrieval.run(
    {"text_embedder": {"text": query}, "bm25_retriever": {"query": query}, "ranker": {"query": query}}
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
retrieved_result["ranker"]["documents"]
retrieved_result_df = pd.DataFrame(retrieved_result["ranker"]["documents"])
retrieved_result_df['query'] = query
retrieved_result_df

,id,content,blob,meta,score,embedding,sparse_embedding,query
0,e686de84900a7b27a7d098eabd3963f222d02b200c0f6d9290373c0824cb52e6,"In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .",None,"{'sentiment': 'positive', 'source_id': 'bca848dda5f80a663b837b5ea06d5487b72541e3c5d3d6b7ed17ebba0f689553', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}",0.010501,"[0.018613317981362343, 0.00785414595156908, 0.025548791512846947, 0.03751637786626816, 0.0609283410012722, -0.03064511902630329, -0.006793426349759102, 0.12092845886945724, 0.01998920924961567, 0.023418040946125984, -0.021318554878234863, -0.02649257518351078, 0.03851957246661186, -0.04824081063270569, -0.01883060485124588, -0.01011818926781416, 0.02987120859324932, -0.06498829275369644, -0.07069810479879379, -0.0528804250061512, -0.00406131986528635, -0.009034822694957256, 0.030583946034312248, 0.035009413957595825, 0.04474963992834091, 0.008541169576346874, -0.018446676433086395, 0.053188178688287735, -0.05401905998587608, -0.11701404303312302, -0.0021511747036129236, -0.05894234776496887, 0.055513896048069, -0.04585089534521103, -0.008870569989085197, -0.042530156672000885, -0.08072...",None,"Can you identify the predictions from the documents? I define a prediction as: A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:\n\n 1. <p_s>\n - Defined as: \n - Source entity that states the <p>\n - Characteristics:\n - A person with either: a name only, profile name only, geneder only, domain specific title only or any combination of these.\n - An associated organization\n - Named entity: Person, organization\n - Part of speech: Noun\n - Examples:\n 1. A person with a name only: Detravious\n 2. A person with a profile name: FitToJesus\n 3. A person with a gender only: He\n ..."
1,00098507dbc5241dc1f90f8635bcbfb77307f21c7d79a6ec18532cb958468aae,"Its board of directors will propose a dividend of EUR0 .12 per share for 2010 , up from the EUR0 .08 per share paid in 2009 .",None,"{'sentiment': 'positive', 'source_id': '6e6bf28594d6da037f1700c81b166474a47d8f5faf4fbbdf61a3b01e8df5f051', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}",0.008239,"[-0.052842069417238235, -0.06388484686613083, -0.0154475262388587, -0.004088420886546373, 0.014476707205176353, -0.013755597174167633, -0.045835498720407486, 0.05932892858982086, -0.02497805655002594, 0.06341962516307831, -0.014904072508215904, 0.03360829874873161, -0.04799428582191467, 0.022012311965227127, -0.014462808147072792, 0.000505294359754771, -0.018619602546095848, -0.027622969821095467, -0.048830777406692505, 0.04618675261735916, 0.03534964099526405, -0.10417553782463074, 0.0666796937584877, -0.022784095257520676, 0.007957450114190578, -0.08822828531265259, -0.009818666614592075, -0.024691887199878693, -0.0494588166475296, -0.09553887695074081, 0.051997438073158264, 0.0007014370057731867, 0.02614424005150795, -0.007209648843854666, 0.02206900343298912, -0.0032518357038497925...",None,"Can you identify the predictions from the documents? I define a prediction as: A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:\n\n 1. <p_s>\n - Defined as: \n - Source entity that states the <p>\n - Characteristics:\n - A person with either: a name only, profile name only, geneder only, domain specific title only or any combination of these.\n - An associated organization\n - Named entity: Person, organization\n - Part of speech: Noun\n - Examples:\n 1. A person with a name only: Detravious\n 2. A person with a profile name: FitToJesus\n 3. A person with a gender only: He\n ..."
2,3ddf930c3b3e084b53333715ff8a9f098e1bce74be70af7934a29c73e68a4870,Net sales increased to EUR193 .3 m from EUR179 .9 m and pretax profit rose by 34.2 % to EUR43 .1 m. ( EUR1 = USD1 .4 ),None,"{'sentiment': 'positive', 'source_id': '05cd57bf4ff16df5b7ce025

## Save Data

In [16]:
"""If save as new file:"""

# path = os.path.join(notebook_dir, "../data/rag/retrieved/")
# prefix = "retrieved_results"
# save_file_type = "csv"
# DataProcessing.save_to_file(retrieved_result_df, path, prefix, save_file_type)\


"""If append to previous file, then save:"""
path = os.path.join(notebook_dir, "../data/rag/retrieved/", "text_label_name_meta_data-v1.csv")
# prefix = "text_label_metadata"
save_file_type = "csv"
df = DataProcessing.load_from_file(path, save_file_type)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df

,text,llm_label,llm_name,meta_data
0,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .,prediction,llama-3.1-8b-instant,"id 084eec7989f0cfbbb94860b54ffffe5c0ea064f896a8dc4e4e69dfa1f4fb6eb0\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': '2c106b768ca3455523070e94cfcdbc9e145ca808ae79f54e5d2f7c56e21ab280', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."
1,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .,Non-prediction,llama-3.3-70b-versatile,"id 084eec7989f0cfbbb94860b54ffffe5c0ea064f896a8dc4e4e69dfa1f4fb6eb0\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': '2c106b768ca3455523070e94cfcdbc9e145ca808ae79f54e5d2f7c56e21ab280', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."
2,"Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .",prediction,llama-3.1-8b-instant,id 53f4afca60c6d97e69fe7bf80b319601dfaed593c...
3,"Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .",Non-prediction,llama-3.3-70b-versatile,id 53f4afca60c6d97e69fe7bf80b319601dfaed593c...
4,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",prediction,llama-3.1-8b-instant,id c8250eb1bfc021b035d71ba172e262a21ab427b75...
5,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",Prediction,llama-3.3-70b-versatile,id c8250eb1bfc021b035d71ba172e262a21ab427b75...
6,"The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .",prediction,llama-3.1-8b-instant,id bc1d4924ecbc2356699cdec2c4662ea3f2125a2dc...
7,"The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .",Non-prediction,llama-3.3-70b-versatile,id bc1d4924ecbc2356699cdec2c4662ea3f2125a2dc...
8,"Finnish Talentum reports its operating profit increased to EUR 20.5 mn in 2005 from EUR 9.3 mn in 2004 , and net sales totaled EUR 103.3 mn , up from EUR 96.4 mn .",prediction,llama-3.1-8b-instant,"id cda0ab041b78439da23a9559adf5a33bc578eb7080be3bf050270d043d491cce\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': 'c4fff1f5fb136e3170ef2af33b26fe54a73fcf7f1f8feb8458dcf57c6fd3d7f8', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."
9,"Finnish Talentum reports its operating profit increased to EUR 20.5 mn in 2005 from EUR 9.3 mn in 2004 , and net sales totaled EUR 103.3 mn , up from EUR 96.4 mn .",Non-prediction,llama-3.3-70b-versatile,"id cda0ab041b78439da23a9559adf5a33bc578eb7080be3bf050270d043d491cce\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': 'c4fff1f5fb136e3170ef2af33b26fe54a73fcf7f1f8feb8458dcf57c6fd3d7f8', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."


In [17]:
content = retrieved_result_df.content
content

0                                                                                     In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .
1                                                                                     Its board of directors will propose a dividend of EUR0 .12 per share for 2010 , up from the EUR0 .08 per share paid in 2009 .
2                                                                                            Net sales increased to EUR193 .3 m from EUR179 .9 m and pretax profit rose by 34.2 % to EUR43 .1 m. ( EUR1 = USD1 .4 )
3    With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .
4                                                                        Incap Contract Manufacturing Services Pvt Ltd , a subsidiary of Incap Corporati

In [18]:
meta_data_df = retrieved_result_df.drop(['content'], axis=1)
meta_data_df.head(1)

,id,blob,meta,score,embedding,sparse_embedding,query
0,e686de84900a7b27a7d098eabd3963f222d02b200c0f6d9290373c0824cb52e6,None,"{'sentiment': 'positive', 'source_id': 'bca848dda5f80a663b837b5ea06d5487b72541e3c5d3d6b7ed17ebba0f689553', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}",0.010501,"[0.018613317981362343, 0.00785414595156908, 0.025548791512846947, 0.03751637786626816, 0.0609283410012722, -0.03064511902630329, -0.006793426349759102, 0.12092845886945724, 0.01998920924961567, 0.023418040946125984, -0.021318554878234863, -0.02649257518351078, 0.03851957246661186, -0.04824081063270569, -0.01883060485124588, -0.01011818926781416, 0.02987120859324932, -0.06498829275369644, -0.07069810479879379, -0.0528804250061512, -0.00406131986528635, -0.009034822694957256, 0.030583946034312248, 0.035009413957595825, 0.04474963992834091, 0.008541169576346874, -0.018446676433086395, 0.053188178688287735, -0.05401905998587608, -0.11701404303312302, -0.0021511747036129236, -0.05894234776496887, 0.055513896048069, -0.04585089534521103, -0.008870569989085197, -0.042530156672000885, -0.08072...",None,"Can you identify the predictions from the documents? I define a prediction as: A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:\n\n 1. <p_s>\n - Defined as: \n - Source entity that states the <p>\n - Characteristics:\n - A person with either: a name only, profile name only, geneder only, domain specific title only or any combination of these.\n - An associated organization\n - Named entity: Person, organization\n - Part of speech: Noun\n - Examples:\n 1. A person with a name only: Detravious\n 2. A person with a profile name: FitToJesus\n 3. A person with a gender only: He\n ..."


In [19]:
new_df = pd.DataFrame(content)
new_df

,content
0,"In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn ."
1,"Its board of directors will propose a dividend of EUR0 .12 per share for 2010 , up from the EUR0 .08 per share paid in 2009 ."
2,Net sales increased to EUR193 .3 m from EUR179 .9 m and pretax profit rose by 34.2 % to EUR43 .1 m. ( EUR1 = USD1 .4 )
3,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .
4,"Incap Contract Manufacturing Services Pvt Ltd , a subsidiary of Incap Corporation of Finland , plans to double its revenues by 2007-2008 ."
5,"Net income from life insurance doubled to EUR 6.8 mn from EUR 3.2 mn , and net income from non-life insurance rose to EUR 5.2 mn from EUR 1.5 mn in the corresponding period in 2009 ."
6,"STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) raised the fair value for shares in four of the largest Nordic forestry groups ."
7,"Consolidated net sales increased 16 % to reach EUR74 .8 m , while operating profit amounted to EUR0 .9 m compared to a loss of EUR0 .7 m in the prior year period ."
8,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .
9,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing ."


In [20]:
meta_datas = []
for idx, row in meta_data_df.iterrows():
    meta_datas.append(row)

meta_datas

[id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 e686de84900a7b27a7d098eabd3963f222d02b200c0f6d9290373c0824cb52e6
 blob                                                                                                                                                                              

In [21]:
new_df['meta_data'] = meta_datas
new_df.rename(columns={'content': 'text'}, inplace=True)
new_df

,text,meta_data
0,"In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .",id e686de84900a7b27a7d098eabd3963f222d02b200...
1,"Its board of directors will propose a dividend of EUR0 .12 per share for 2010 , up from the EUR0 .08 per share paid in 2009 .",id 00098507dbc5241dc1f90f8635bcbfb77307f21c7...
2,Net sales increased to EUR193 .3 m from EUR179 .9 m and pretax profit rose by 34.2 % to EUR43 .1 m. ( EUR1 = USD1 .4 ),id 3ddf930c3b3e084b53333715ff8a9f098e1bce74b...
3,With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .,id 9e8da179f1923e36a0092be7b7b424377e1daae08...
4,"Incap Contract Manufacturing Services Pvt Ltd , a subsidiary of Incap Corporation of Finland , plans to double its revenues by 2007-2008 .",id 9fc8c8f0070f5991c474874ace926849724c41842...
5,"Net income from life insurance doubled to EUR 6.8 mn from EUR 3.2 mn , and net income from non-life insurance rose to EUR 5.2 mn from EUR 1.5 mn in the corresponding period in 2009 .",id c9ed0b12181b55b02d6fe4c14be091b69ec548430...
6,"STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) raised the fair value for shares in four of the largest Nordic forestry groups .",id b0426be8586b1363cc40a132d93ecb6fa082d7403...
7,"Consolidated net sales increased 16 % to reach EUR74 .8 m , while operating profit amounted to EUR0 .9 m compared to a loss of EUR0 .7 m in the prior year period .",id 56b3589d22f351cc29abbff1be37d7353f5a20a10...
8,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .,id 084eec7989f0cfbbb94860b54ffffe5c0ea064f89...
9,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",id c8250eb1bfc021b035d71ba172e262a21ab427b75...


In [22]:
updated_df = pd.concat([df, new_df])
updated_df

,text,llm_label,llm_name,meta_data
0,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .,prediction,llama-3.1-8b-instant,"id 084eec7989f0cfbbb94860b54ffffe5c0ea064f896a8dc4e4e69dfa1f4fb6eb0\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': '2c106b768ca3455523070e94cfcdbc9e145ca808ae79f54e5d2f7c56e21ab280', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."
1,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .,Non-prediction,llama-3.3-70b-versatile,"id 084eec7989f0cfbbb94860b54ffffe5c0ea064f896a8dc4e4e69dfa1f4fb6eb0\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': '2c106b768ca3455523070e94cfcdbc9e145ca808ae79f54e5d2f7c56e21ab280', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."
2,"Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .",prediction,llama-3.1-8b-instant,id 53f4afca60c6d97e69fe7bf80b319601dfaed593c...
3,"Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .",Non-prediction,llama-3.3-70b-versatile,id 53f4afca60c6d97e69fe7bf80b319601dfaed593c...
4,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",prediction,llama-3.1-8b-instant,id c8250eb1bfc021b035d71ba172e262a21ab427b75...
5,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .",Prediction,llama-3.3-70b-versatile,id c8250eb1bfc021b035d71ba172e262a21ab427b75...
6,"The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .",prediction,llama-3.1-8b-instant,id bc1d4924ecbc2356699cdec2c4662ea3f2125a2dc...
7,"The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .",Non-prediction,llama-3.3-70b-versatile,id bc1d4924ecbc2356699cdec2c4662ea3f2125a2dc...
8,"Finnish Talentum reports its operating profit increased to EUR 20.5 mn in 2005 from EUR 9.3 mn in 2004 , and net sales totaled EUR 103.3 mn , up from EUR 96.4 mn .",prediction,llama-3.1-8b-instant,"id cda0ab041b78439da23a9559adf5a33bc578eb7080be3bf050270d043d491cce\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': 'c4fff1f5fb136e3170ef2af33b26fe54a73fcf7f1f8feb8458dcf57c6fd3d7f8', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."
9,"Finnish Talentum reports its operating profit increased to EUR 20.5 mn in 2005 from EUR 9.3 mn in 2004 , and net sales totaled EUR 103.3 mn , up from EUR 96.4 mn .",Non-prediction,llama-3.3-70b-versatile,"id cda0ab041b78439da23a9559adf5a33bc578eb7080be3bf050270d043d491cce\nblob NaN\nmeta {'sentiment': 'positive', 'source_id': 'c4fff1f5fb136e3170ef2af33b26fe54a73fcf7f1f8feb8458dcf57c6fd3d7f8', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}\nscore ..."


In [23]:
path = os.path.join(notebook_dir, "../data/rag/retrieved/")
prefix = "text_label_name_meta_data"
save_file_type = "csv"
DataProcessing.save_to_file(updated_df, path, prefix, save_file_type)

Saved to: 
	/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/rag/retrieved/text_label_name_meta_data-v1.csv


### Pretty Print the Results
Create a function to print a kind of *search page*.

In [ ]:
def pretty_print_results(prediction):
    for doc in prediction["documents"]:
        print(doc.content, "\t", doc.score)
        # print(doc.meta["abstract"])
        print("\n", "\n")

In [ ]:
pretty_print_results(result["ranker"])

## What's next

🎉 Congratulations! You've create a hybrid retrieval pipeline!

If you'd like to use this retrieval method in a RAG pipeline, check out [Tutorial: Creating Your First QA Pipeline with Retrieval-Augmentation](https://haystack.deepset.ai/tutorials/27_first_rag_pipeline) to learn about the next steps.

To stay up to date on the latest Haystack developments, you can [sign up for our newsletter](https://landing.deepset.ai/haystack-community-updates) or [join Haystack discord community](https://discord.gg/haystack).

Thanks for reading!